## Importing modules

In [1]:
import metapy
import csv
import requests
import re
from bs4 import BeautifulSoup,Comment
from selenium import webdriver
import time
import json
import nltk
import pandas as pd
import gensim
import spacy
import en_core_web_sm 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import numpy as np
import lda.datasets
import sklearn.feature_extraction.text as text
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import pairwise_distances_argmin
from tkinter import *
import webbrowser

C:\Users\garim\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Reading the database

In [2]:
df = pd.read_csv('final_worked_new.csv',encoding='cp1252').dropna()
df.head()

,Year,Term,Course_ID,Course_Name,Course_Description
0,2018.0,Fall,AE 100,Intro to Aerospace Engineering,Aerospace Engineering Course.Introduction to t...
1,2018.0,Fall,AE 199,Undergraduate Open Seminar,Aerospace Engineering Course.This is a seminar...
2,2018.0,Fall,AE 202,Aerospace Flight Mechanics,Aerospace Engineering Course.Fundamental princ...
3,2018.0,Fall,AE 298,Special Topics,Aerospace Engineering Course.Lectures and disc...
4,2018.0,Fall,AE 311,Incompressible Flow,Aerospace Engineering Course.Equations of moti...


## Extracting data for training LDA

In [3]:
data = df.Course_Description.values.tolist()

## Cleaning the list

In [4]:
data = [re.sub("\'", "", sent) for sent in data]

## Functions required for bag of words and lemmatization

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#### The above gensim.utils.simple_preprocess additional information can be found here : https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html. It is used to convert a document into list of tokens. It lowercases the letters, tokenizes them and if deacc property is set to TRUE removes punctuations 

In [6]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        #print(sent)
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        #texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        #print(texts_out)
    return texts_out

#### Lemmatization is an advanced form of stemming that converts words into their parent word using inputs like noun, adjective, verb and adverb. The default tag is noun. Using this we can find the root word by their characteristics. For example running will become run using lemmatization.

## Applying tokenizer and lemmatizer to the list of documents

In [7]:
data_words = list(sent_to_words(data))
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\Users\garim\Anaconda3\lib\site-pa

C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
C:\Users\garim\Anaconda3\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])


## Stopwords definition

In [8]:
stop_no = open("numbers.txt")
list_no=[]
for line in stop_no : 
    list_no.append(re.sub(r'[-,?!":;()|]\n',"",line,re.UNICODE).replace("'", "").strip())
STOPLIST = list(set(stopwords.words('english') + list_no + ["new","experience","edu","illinois","advise","advice","topic","area","approval","use","consent","letter","grading","term","repeat","provide","require","need","include","study","approve","vary","exist","consider","make","prerequisite","credit", "course","hour","hours","courses","student","students","instructor","instructors"]+list(ENGLISH_STOP_WORDS)))
STOPLIST.extend(('.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','/','-'))

#### Very common words can be fatal to machine learning and text mining analysis and hence we define stopwords. This stopwords list is then put into a count vectorizer which removes these stopwords from the list of documents

## Count Vectorizer definition

In [9]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        
                             stop_words=STOPLIST,             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',              
                            )

#### CountVectorizer function converts a collection of documents into token counts. The documentation for the same can be found here : http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [10]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

## Observing feature names in the vectorizer

In [11]:
vectorizer.get_feature_names()

['able',
 'academic',
 'access',
 'accountancy',
 'accounting',
 'accy',
 'acquisition',
 'activity',
 'additional',
 'address',
 'administration',
 'admission',
 'advanced',
 'aerospace',
 'affect',
 'algorithm',
 'alternative',
 'analysis',
 'analytic',
 'analytical',
 'analyze',
 'application',
 'apply',
 'approach',
 'appropriate',
 'art',
 'aspect',
 'asset',
 'assignment',
 'associate',
 'augment',
 'background',
 'base',
 'basic',
 'beam',
 'behavior',
 'bioengineer',
 'bioengineering',
 'biological',
 'biology',
 'book',
 'boundary',
 'build',
 'building',
 'business',
 'calculus',
 'campus',
 'capital',
 'career',
 'case',
 'cell',
 'challenge',
 'change',
 'characteristic',
 'chem',
 'chemical',
 'chemistry',
 'circuit',
 'civil',
 'class',
 'classical',
 'classification',
 'cluster',
 'collection',
 'college',
 'communication',
 'community',
 'company',
 'complete',
 'completion',
 'complex',
 'component',
 'composition',
 'computation',
 'computational',
 'compute',
 'compu

In [12]:
vocab = np.array(vectorizer.get_feature_names()) ##Making vocabulary in python

## LDA model!

#### To chose the best LDA model we can use GridSearchCV. This would give us the best model to use in between n_topics and learning decays

In [13]:
search_params = {'n_components': [10, 15,20], 'learning_decay': [.5, .7, .9]}
lda_model = LatentDirichletAllocation(random_state=100,n_topics=10)
model = GridSearchCV(lda_model, param_grid=search_params)
model.fit(data_vectorized)

C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\skl

C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\garim\Anaconda3\lib\site-packages\skl

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=10,
             perp_tol=0.1, random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_components': [10, 15, 20], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## Selecting the Best LDA model

In [14]:
best_lda_model = model.best_estimator_
print("Chosen LDA Model is: ", model.best_params_)
lda_output = best_lda_model.transform(data_vectorized)

Chosen LDA Model is:  {'learning_decay': 0.9, 'n_components': 10}


#### This model has already been fitted to the vectorizer as can be seen above. Hence our LDA model with best features is ready!!

## Visualization

#### To analyze how well our topics are divided we use the famous pyLDAvis package.This helps to give us intricate details about our model and its classifications. This documentation can be found here: http://pyldavis.readthedocs.io/en/latest/

In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      16.709447        1       1  117.666664  108.519386
2      15.958059        1       2  154.668869 -139.476181
3      14.604659        1       3 -107.375504 -122.336288
7      13.203476        1       4 -114.309425   16.108088
8      11.238926        1       5  -26.279861  145.703339
6      10.523399        1       6  220.311066    3.861347
0       8.821348        1       7   30.628839 -199.284836
9       5.140425        1       8  106.770439  -26.802650
5       2.583445        1       9    8.052059   33.491005
4       1.216815        1      10    5.157754  -82.241348, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
141   Default  791.000000     engineering  791.000000  30.0000  30.0000
44    Default  266.000000        business  266.000000  29.0000  29.0000
389   Default  501.000000         science  501.000000  28.0000  28.0000
65    Default  285.000000   communication  285.000000  27.0000  27.0000
131   Default  250.000000      electronic  250.000000  26.0000  26.0000
76    Default  319.000000        computer  319.000000  25.0000  25.0000
100   Default  245.000000           datum  245.000000  24.0000  24.0000
218   Default  286.000000     information  286.000000  23.0000  23.0000
166   Default  119.000000         finance  119.000000  22.0000  22.0000
293   Default   69.000000         nuclear   69.000000  21.0000  21.0000
10    Default  128.000000  administration  128.000000  20.0000  20.0000
275   Default  141.000000      mechanical  141.000000  19.0000  19.0000
323   Default  136.000000         physics  136.000000  18.0000  18.0000
326   Default   46.000000          plasma   46.000000  17.0000  17.0000
13    Default  106.000000       aerospace  106.000000  16.0000  16.0000
3     Default   56.000000     accountancy   56.000000  15.0000  15.0000
5     Default   52.000000            accy   52.000000  14.0000  14.0000
360   Default   39.000000    radiological   39.000000  13.0000  13.0000
103   Default  101.000000      department  101.000000  12.0000  12.0000
267   Default  130.000000            math  130.000000  11.0000  11.0000
167   Default   73.000000       financial   73.000000  10.0000  10.0000
84    Default   92.000000    construction   92.000000   9.0000   9.0000
428   Default   91.000000       statistic   91.000000   8.0000   8.0000
259   Default  124.000000      management  124.000000   7.0000   7.0000
350   Default   71.000000         program   71.000000   6.0000   6.0000
142   Default   41.000000      enrollment   41.000000   5.0000   5.0000
349   Default  118.000000    professional  118.000000   4.0000   4.0000
282   Default  130.000000           model  130.000000   3.0000   3.0000
246   Default   70.000000         library   70.000000   2.0000   2.0000
4     Default   28.000000      accounting   28.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
228   Topic10    1.740666   investigation   17.645051   2.0927  -4.8807
25    Topic10    1.008689             art   11.569793   1.9692  -5.4264
359   Topic10    2.135309       radiation   26.595624   1.8868  -4.6764
357   Topic10    1.667449    quantitative   23.074039   1.7815  -4.9237
375   Topic10    1.333623          report   18.513443   1.7783  -5.1471
370   Topic10    2.344729          relate   33.812312   1.7403  -4.5828
303   Topic10    1.457381       operation   21.765731   1.7052  -5.0584
237   Topic10    1.953020      laboratory   30.501796   1.6605  -4.7656
120   Topic10    1.546174      discussion   25.917617   1.5898  -4.9992
250   Topic10    1.106737      literature   19.739639   1.5277  -5.3336
214   Topic10    2.220710      individual   39.803573   1.5228  -4.6372
174   Topic10    1.043629           force   19.929964   1.4594  -5.3923
50    Topic10    1.30

#### To investigate further we can print probabilities of the LDA model per Topic. This can be seen below.

In [16]:
topic_keywords = pd.DataFrame(best_lda_model.components_)
topic_names = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
topic_keywords.columns = vectorizer.get_feature_names()
topic_keywords.index = topic_names

topic_keywords

,able,academic,access,accountancy,accounting,accy,acquisition,activity,additional,address,...,visualization,water,wave,way,web,work,world,write,writing,year
Topic0,0.138781,0.183117,0.152868,0.214326,0.173065,0.180842,0.145121,0.154280,0.151145,0.146955,...,0.191011,4.096251,1.018265,0.151208,0.851960,2.140356,0.199293,0.159479,0.452996,0.165714
Topic1,0.151885,18.275117,14.563248,1.269699,0.612096,0.734374,5.858323,11.642390,5.180503,1.105621,...,0.166526,3.180184,0.150405,18.455940,17.749721,24.317451,16.589375,13.430963,29.795139,0.494686
Topic2,3.745402,0.734920,3.709780,0.222807,0.180338,0.182239,0.137420,2.168063,1.966314,1.681309,...,0.193265,0.930266,0.258504,0.223478,3.340019,17.161929,4.727434,3.544350,8.368015,0.313594
Topic3,0.167210,0.149265,0.186799,0.167053,0.139834,0.173595,0.182069,0.152196,0.688202,0.180482,...,1.139777,10.824552,6.307171,0.150821,0.274161,3.229569,1.643807,1.615067,0.453466,0.189281
Topic4,0.144508,0.139705,0.144280,0.217326,0.141739,0.177302,0.149053,0.158874,0.151313,0.157721,...,0.150595,0.152102,0.149765,0.145783,0.144998,0.145155,0.142388,0.146247,0.148422,0.143110
Topic5,0.146006,0.148806,0.148112,31.723563,15.995476,26.845320,0.174520,0.164542,0.662121,0.480991,...,0.154420,0.150885,0.151218,0.158114,0.139941,0.898313,0.142131,0.155457,0.152959,0.144067
Topic6,1.681101,3.111710,0.140372,4.557555,4.095597,9.004562,1.535132,4.250848,2.313645,3.649300,...,0.161126,0.144203,0.141073,3.572123,0.149886,21.309000,11.364097,0.176969,0.182725,4.950635
Topic7,5.979701,1.056762,0.153671,1.479576,0.736161,0.637822,0.421859,0.574087,0.171175,2.261466,...,36.850993,0.160552,0.142051,3.763918,24.148075,16.296166,0.238222,0.247762,0.199329,0.154259
Topic8,0.180180,0.146139,1.286264,0.196953,0.157487,0.192880,4.737991,0.412475,0.213388,0.170158,...,0.565546,0.146650,19.346747,0.155930,0.195107,0.304455,0.333085,3.239195,0.497259,1.960816
Topic9,0.147413,6.824665,0.149782,4.862481,0.570949,3.385597,2.354425,0.209433,0.159512,0.166012,...,0.173454,0.318313,0.150159,2.279638,0.169716,0.255578,0.347993,0.378368,0.233239,4.841503


## Predicting Courses that Match a specific Query

#### Lets dive into predicting the list of topics based on the input by the user. For this we need to define the predict function.

In [17]:
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
    tokens = list(sent_to_words(text))
    lemm = lemmatization(tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    vector = vectorizer.transform(lemm)
    topic_probability_scores = best_lda_model.transform(vector)
    topic = topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores


#### Defining a similarity function for getting the top 20 course names

In [18]:
def similar_documents(text, doc_topic_probs, documents = df, nlp=nlp, top_n=20):
    
    t1 = "".join(text)
    text = []
    text.append(t1)
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    course_name = []
    course_id = []
    dot_product = doc_ids.tolist()
    for i in dot_product : 
        course_id.append(df["Course_ID"][i])
        course_name.append(df["Course_Name"][i])
    
    final_rank = list(zip(course_id, course_name))
    searchf=Tk()
    
    searchf.configure(background="sienna1")
    str_list = " ".join(text)
    searchf.wm_title("Course Search")
    searchf.wm_iconbitmap("images/uiuc_logo.ico")
    blank='           '
    blanklabel=Label( searchf,text=blank*40,font=("ComicSansMS", 10),bg="sienna1")
    label1 = Label( searchf,text="Search Results for \"" + str_list+"\"",font=("ComicSansMS", 20),bg="sienna1")
    label1.pack()
    blanklabel.pack()
    def callbacks(event):
        webbrowser.open_new(r"https://courses.illinois.edu/schedule/2018/fall")
    
    b = Button(searchf,text="Go to Course Page!",anchor=W, justify=LEFT, padx=20,font=("ComicSansMS", 10))
    b.bind("<Button-1>", callbacks)
    b.pack()
    blanklabel=Label( searchf,text=blank*40,font=("ComicSansMS", 10),bg="sienna1")
    blanklabel.pack()
    #scrollbar.pack(side=RIGHT, fill=Y)
    for i in final_rank[:]:
        labl=Label( searchf,text=i,font=("ComicSansMS", 12),justify=LEFT, fg="blue", cursor="hand2",bg="sienna1")
        labl.pack()
        

## Using Tkinter to take value from USER

In [ ]:
try:
    top= Tk()
except:
    top=Toplevel()
    top= Tk()
top.wm_title("Course Search")
photo = PhotoImage(file="images/uiuc_heading.gif")
dimensions = "image size: %dx%d" % (photo.width(), photo.height())
w = Label(top,image=photo)
w.pack(side = "top")
photoimage = PhotoImage(file="images/Alma_mater.gif")
v = Label(top, image=photoimage)

v.pack(side = "bottom", fill = "both")
top.wm_iconbitmap("images/uiuc_logo.ico")
f = Frame(top, width=600,height=350,bg="royal blue")
f.pack(fill=X, expand=True)
e1=Entry(top,bd=10,width=40,font=("Helvetica", 15))
e1.insert(END, 'search here')
e1.place(relx=0.5, rely=0.35, anchor=CENTER)
text = e1.get()

b1=Button(top,font=("Helvetica", 15),text="Click me!",command= lambda:similar_documents(text=list(e1.get()), doc_topic_probs=lda_output, documents = df, top_n=20))
b1.place(relx=0.5, rely=0.5, anchor=CENTER)
top.mainloop()